**Visualization_Project**

Ariel Siman Tov 209499821 | Tal Klein 209234203

# pip install

In [1]:
pip install nba_api

In [2]:
pip install Dash

In [3]:
pip install dash_daq

In [4]:
pip install dash-bootstrap-components

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 4.0 MB/s eta 0:00:00


# import

In [5]:
import pandas as pd
from nba_api.stats.endpoints import playercareerstats
import pandas as pd
import numpy as np
import dash
from dash import dcc, html
import plotly.express as px
import plotly.graph_objs as go
from dash.dependencies import Input, Output
import logging
from google.colab import output
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.endpoints import LeagueDashPlayerClutch
from nba_api.stats.endpoints import playerdashboardbyyearoveryear
import base64
import regex as re




# get data

## data for bar / spider plot

In [6]:
def get_player_career_stats(player_name):
    # Get player ID
    player_dict = players.get_players()
    player = [p for p in player_dict if p['full_name'] == player_name]
    if not player:
        raise ValueError(f"Player '{player_name}' not found.")
    player_id = player[0]['id']

    # Get basic career stats
    basic_stats_df = playerdashboardbyyearoveryear.PlayerDashboardByYearOverYear(player_id=player_id)
    basic_stats_df = basic_stats_df.get_data_frames()[1]

    # Specify the columns to keep for basic stats
    columns_to_keep_basic = [
        'GROUP_VALUE', 'GP','FGM', 'FGA','FG3_PCT', 'FTM', 'FTA',
        'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
        'PF', 'PFD', 'PTS'
    ]

    # Filter the basic stats DataFrame to keep only the specified columns
    filtered_basic_stats_df = basic_stats_df[columns_to_keep_basic].copy()

    # Extract the year from GROUP_VALUE and add it as a new column
    filtered_basic_stats_df['Season'] = filtered_basic_stats_df['GROUP_VALUE'].apply(lambda x: int(x[:4]))

    # Get advanced career stats
    adv_stats_df = playerdashboardbyyearoveryear.PlayerDashboardByYearOverYear(player_id=player_id, measure_type_detailed='Advanced')
    adv_stats_df = adv_stats_df.get_data_frames()[1]

    # Specify the columns to keep for advanced stats
    columns_to_keep_adv = [
        'GROUP_VALUE', 'MIN', 'OFF_RATING', 'DEF_RATING',
         'AST_PCT', 'AST_TO', 'EFG_PCT', 'TS_PCT', 'USG_PCT', 'PACE',
        'PIE', 'POSS'
    ]

    # Filter the DataFrame to keep only the specified columns
    filtered_adv_stats_df = adv_stats_df[columns_to_keep_adv].copy()

    # Extract the year from GROUP_VALUE and add it as a new column
    filtered_adv_stats_df['Season'] = filtered_adv_stats_df['GROUP_VALUE'].apply(lambda x: int(x.split('-')[0]))

    # Function to convert year to season format
    def convert_to_season(year):
        return f"{year}-{str(year + 1)[-2:]}"

    # Apply the conversion function to the 'Season' column
    filtered_basic_stats_df['Season'] = filtered_basic_stats_df['Season'].apply(convert_to_season)
    filtered_adv_stats_df['Season'] = filtered_adv_stats_df['Season'].apply(convert_to_season)

    # Merge the DataFrames on the 'Season' column
    player_stats = pd.merge(filtered_basic_stats_df, filtered_adv_stats_df, on='Season', how='inner')

    seasons = ["2023-24", "2022-23", "2021-22","2020-21"]
    if player_name == 'Omri Casspi':
        seasons = ["2009-10", "2010-11", "2011-12","2012-13"]


    # List to store individual DataFrames for each season
    dfs = []

    # Iterate over each season
    for season in seasons:
        # Instantiate the LeagueDashPlayerClutch endpoint for the current season
        clutch_stats = LeagueDashPlayerClutch(clutch_time="Last 30 Seconds", season=[season])

        # Fetch the data
        clutch_data = clutch_stats.get_data_frames()[0]  # Get the first DataFrame from the response

        # Filter clutch data for LeBron James
        lebron_clutch_stats = clutch_data[clutch_data['PLAYER_ID'] == player_id].copy()  # Use .copy() to avoid SettingWithCopyWarning

        # Add season column using .loc to avoid SettingWithCopyWarning
        lebron_clutch_stats.loc[:, 'Season'] = season

        # Append DataFrame to list
        dfs.append(lebron_clutch_stats)

    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(dfs, ignore_index=True)

    # Add PTS_RANK column from clutch stats to player_stats DataFrame
    player_stats = pd.merge(player_stats, combined_df[['Season', 'PTS_RANK','FG_PCT_RANK']], on='Season', how='left')

    return player_stats

# Example usage for Deni Avdija
deni_stats = get_player_career_stats('Deni Avdija')
# Example usage for LeBron James
lebron_stats = get_player_career_stats('LeBron James')
# Example usage for Omri Casspi
casspi_stats = get_player_career_stats('Omri Casspi')
casspi_stats.loc[21:24, 'Season'] = ['2023-24', '2022-23', '2021-22', '2020-21']

deni_stats.to_csv('deni_stats.csv', index=False)
lebron_stats.to_csv('lebron_stats.csv', index=False)
casspi_stats.to_csv('casspi_stats.csv', index=False)


In [7]:
df_15_columns = deni_stats.iloc[:, :16]
df_15_columns

,GROUP_VALUE_x,GP,FGM,FGA,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,PF
0,2023-24,75,406,803,0.374,202,273,0.740,84,456,540,286,160,59,36,185
1,2022-23,76,253,579,0.297,122,165,0.739,74,413,487,211,123,65,29,209
2,2021-22,82,250,579,0.317,103,136,0.757,52,373,425,167,87,59,44,190
3,2020-21,54,130,312,0.315,29,45,0.644,22,240,262,63,33,32,15,140


## data for court plot

In [8]:
# Function to get player ID from name
def get_player_id(player_name):
    player_dict = players.get_players()
    for player in player_dict:
        if player['full_name'].lower() == player_name.lower():
            return player['id']
    return None

# Fetch Deni Avdija's player ID
player_name = "Deni Avdija"
player_id = get_player_id(player_name)
if player_id is None:
    print(f"Player {player_name} not found.")
else:
    # Seasons to fetch data for
    seasons = ["2020-21", "2021-22", "2022-23", "2023-24"]
    all_seasons_data = []

    for season in seasons:
        # Fetch the shooting data for each season
        shotchart = shotchartdetail.ShotChartDetail(
            team_id=0,  # Use 0 to fetch data for all teams he played for
            player_id=player_id,
            season_type_all_star="Regular Season",
            season_nullable=season,
            context_measure_simple="FGA"
        )

        # Get the shot chart data and metadata
        shot_chart_data = shotchart.get_data_frames()[0]
        shot_chart_data['SEASON'] = season  # Add a column for the season
        all_seasons_data.append(shot_chart_data)

    # Combine all seasons data into a single DataFrame
    DA_shooting_stats = pd.concat(all_seasons_data, ignore_index=True)

    # Filter out shots taken above half-court
    DA_shooting_stats = DA_shooting_stats[DA_shooting_stats['SHOT_DISTANCE'] <= 42]

    column_to_keep = ['SEASON', 'SHOT_DISTANCE', 'SHOT_MADE_FLAG','LOC_X','LOC_Y','SHOT_TYPE','SHOT_ZONE_BASIC','ACTION_TYPE']
    DA_shooting_stats = DA_shooting_stats[column_to_keep]
    # # Save to a CSV file
    DA_shooting_stats.to_csv('DA_shooting_stats.csv', index=False)

## data for defence / offence plot

In [9]:
# from nba_api.stats.endpoints import TeamGameLog, PlayerGameLog, BoxScoreAdvancedV3
# from nba_api.stats.static import teams, players
# import pandas as pd

# # Get Washington Wizards team ID
# wizards = [team for team in teams.get_teams() if team['full_name'] == 'Washington Wizards'][0]
# team_id = wizards['id']

# # Get Danny Avdija's player ID
# avdija = [player for player in players.get_players() if player['full_name'] == 'Deni Avdija'][0]
# player_id = avdija['id']

# seasons = ['2020-21', '2021-22', '2022-23', '2023-24']

# all_games = []

# for season in seasons:
#     game_number = 1  # Reset game number for each season
#     # Get team game log
#     team_games = TeamGameLog(team_id=team_id, season=season).get_data_frames()[0]

#     # Get player game log (basic stats)
#     player_games_basic = PlayerGameLog(player_id=player_id, season=season).get_data_frames()[0]
#     print(team_games)
#     for _, team_game in team_games.iterrows():
#         game_date = team_game['GAME_DATE']
#         print(game_number)

#         # Find matching player game in basic stats
#         player_game_basic = player_games_basic[player_games_basic['GAME_DATE'] == game_date]

#         if not player_game_basic.empty:
#             game_stats_basic = player_game_basic.iloc[0].to_dict()
#         else:
#             game_stats_basic = {col: 0 for col in player_games_basic.columns if col not in ['SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE']}

#         # Get player advanced box score for the game
#         box_score_advanced = BoxScoreAdvancedV3(game_id=team_game['Game_ID']).get_data_frames()[0]
#         print(box_score_advanced)
#         # Find matching player game in advanced stats
#         player_game_advanced = box_score_advanced[box_score_advanced['personId'] == player_id]

#         if not player_game_advanced.empty:
#             game_stats_advanced = player_game_advanced.iloc[0].to_dict()
#         else:
#             game_stats_advanced = {col: 0 for col in box_score_advanced.columns if col not in ['SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE']}

#         # Merge basic and advanced stats for the game
#         game_stats = {**game_stats_basic, **game_stats_advanced}

#         game_stats['GAME_DATE'] = game_date
#         game_stats['SEASON'] = season
#         game_stats['GAME_NUMBER'] = game_number

#         all_games.append(game_stats)
#         game_number += 1

# # Create DataFrame from all games
# df = pd.DataFrame(all_games)

# # Reorder columns to have GAME_NUMBER first
# cols = ['GAME_NUMBER', 'SEASON', 'GAME_DATE'] + [col for col in df.columns if col not in ['GAME_NUMBER', 'SEASON', 'GAME_DATE']]
# DA_defence_offence_df = df[cols]

# column_to_keep = ['GAME_NUMBER', 'SEASON','STL', 'BLK','offensiveRating','defensiveRating','pace','trueShootingPercentage','effectiveFieldGoalPercentage']
# DA_defence_offence_df = DA_defence_offence_df[column_to_keep]
# DA_defence_offence_df.to_csv('DA_defence_offence_df.csv', index=False)

In [10]:
DA_defence_offence_df = pd.read_csv('deni_avdija_game_stats.csv')
DA_defence_offence_df['effectiveFieldGoalPercentage'] =  DA_defence_offence_df['effectiveFieldGoalPercentage']*100
DA_defence_offence_df['trueShootingPercentage'] =  DA_defence_offence_df['trueShootingPercentage']*100
DA_defence_offence_df['defensiveRating'] =  DA_defence_offence_df['defensiveRating']/100

# helper function

## function for bar plor / spider plot

In [11]:
parameters = ['Clutchness','Shooting', 'Scoring', 'Offensive Aggressiveness', 'Play-making', 'Go-to guy','Defensive Aggressiveness']

def GMSCR(PTS,FG,FGA,FTA,FT,ORB,DRB,STL,AST,BLK,PF,TOV):
  return PTS + (0.4 * FG) - (0.7 * FGA) - (0.4 * (FTA - FT)) + (0.7 * ORB) + (0.3 * DRB) + STL + (0.7 * AST) + (0.7 * BLK) - (0.4 * PF) - TOV

def get_data(filtered_stats):
  # Calculate parameters
  gmscr = GMSCR(filtered_stats['PTS'],filtered_stats['FGM'],filtered_stats['FGA'],filtered_stats['FTA'],filtered_stats['FTM'],filtered_stats['OREB'],filtered_stats['DREB'],filtered_stats['STL'],filtered_stats['AST'],filtered_stats['BLK'],filtered_stats['PF'],filtered_stats['TOV'])
  filtered_stats['Play-making'] = ((filtered_stats['AST_TO']/4.5))*100*0.6 + filtered_stats['AST_PCT']*100*0.4
  filtered_stats['Defensive Aggressiveness'] = ((filtered_stats['DEF_RATING']/125)*100)*0.6 +((filtered_stats['STL']/filtered_stats['GP']/2.5)*100)*0.1 + ((filtered_stats['BLK']/filtered_stats['GP']/3)*100)*0.1 + ((filtered_stats['PF']/filtered_stats['GP']/3.5)*100)*0.2
  filtered_stats['Scoring'] = (filtered_stats['PTS']/filtered_stats['GP']/35)*100 *0.5+ gmscr/filtered_stats['GP']/26*100*0.5
  filtered_stats['Go-to guy'] = ((filtered_stats['MIN']/37)*100)*0.25 + ((filtered_stats['USG_PCT'] / 0.35)*100)*0.25+ ((filtered_stats['POSS']/ 5800)*100)*0.25+ ((filtered_stats['PIE'] / 0.23)*100)*0.25
  filtered_stats['Offensive Aggressiveness'] = ((filtered_stats['PFD']/500)*100)*0.3 + ((filtered_stats['OFF_RATING']/125)*100)*0.4+((filtered_stats['PACE']/120)*100)*0.3
  filtered_stats['Shooting'] = filtered_stats['EFG_PCT']*0.2*100 + filtered_stats['TS_PCT']*0.2*100 + filtered_stats['FT_PCT']*0.3*100 + filtered_stats['FG3_PCT']*0.3*100
  filtered_stats['Clutchness'] = (1-(filtered_stats['PTS_RANK'])/242)*100*0.7 + (1-(filtered_stats['FG_PCT_RANK'])/242)*100*0.3
  return filtered_stats


## function for court plot

In [12]:
def ellipse_arc(x_center=0.0, y_center=0.0, a=10.5, b=10.5, start_angle=0.0, end_angle=2 * np.pi, N=200, closed=False):
        t = np.linspace(start_angle, end_angle, N)
        x = x_center + a * np.cos(t)
        y = y_center + b * np.sin(t)
        path = f'M {x[0]}, {y[0]}'
        for k in range(1, len(t)):
            path += f'L{x[k]}, {y[k]}'
        if closed:
            path += ' Z'
        return path
def create_quadratic_bezier_curve(x0, y0, x1, y1, curvature):
    mx = (x0 + x1) / 2
    my = (y0 + y1) / 2
    cx = mx + curvature * (y1 - y0) / 2
    cy = my - curvature * (x1 - x0) / 2
    return f"M {x0}, {y0} Q {cx}, {cy} {x1}, {y1}"

def create_zones():
    zones = [
        ######## three point right ######
        dict(type="path",
              path=ellipse_arc(a=237.5, b=237.5, start_angle=0.386283101, end_angle=np.pi/1.95 - 0.386283101)+' L 80,350'+' L 250,350'+'L 250, 89.47765084'),
        ####### three point left ########
        dict(type="path",
              path=ellipse_arc(a=-237.5, b=237.5, start_angle=0.386283101, end_angle=np.pi/1.95 - 0.386283101) +' L -80,350'+' L -250,350'+'L -250, 89.47765084'),
        ##### three point center ######
    dict(
        type="path",
        path=create_quadratic_bezier_curve(
            215.5 * np.cos(np.pi/2 - 0.386283101), 241 * np.sin(np.pi/2 - 0.386283101),
            -205.5 * np.cos(np.pi/2 - 0.386283101), 241 * np.sin(np.pi/2 - 0.386283101),
            curvature=0.3  # Adjust curvature as needed
        ) +' L -80,350' + ' L 80,350'+' L 80,224'
    ),
        ##### three point right corner ######
        dict(
            type="path",path="M -220,-52.5 L -220, 89.47765084 L -250, 89.47765084 L -250, -52.5 L -220, -52.5 ",layer="below"
        ),
        ##### three point left corner ######
       dict(
            type="path",path="M 220,-52.5 L 220, 89.47765084 L 250, 89.47765084 L 250, -52.5 L 220, -52.5 ",layer="below"
        ),
        ##### two point right corner ######
        dict(
            type="path",path="M 220,-52.5 L 80, -52.5  L 80, 89.47765084  L 220, 89.47765084 L 220,-52.5",layer="below"
        ),
        ##### two point left corner ######
        dict(
            type="path",path="M -220,-52.5 L -80, -52.5  L -80, 89.47765084  L -220, 89.47765084 L -220,-52.5",layer="below"
        ),
        ##### two point center ######
        dict(
            type="path",path=create_quadratic_bezier_curve(
            215.5 * np.cos(np.pi/2 - 0.386283101), 241.5 * np.sin(np.pi/2 - 0.386283101),
            -215.5 * np.cos(np.pi/2 - 0.386283101), 241.5 * np.sin(np.pi/2 - 0.386283101),
            curvature=0.3  # Adjust curvature as needed
        ) + 'L -80, 89.47765084 L 80, 89.47765084 L 80, 224',layer="below"
        ),
        ##### in paint ######
        dict(
            type="path",path="M -80,-52.5 L 80, -52.5  L 80, 89.47765084  L -80, 89.47765084 L -80, -52.5 ",layer="below"
        ),
        ##### two point right ######
        dict(
            type="path",path="M 220,89.47765084 L 80, 89.47765084  L 80, 224 "+ellipse_arc(a=237.5, b=237.5, start_angle=0.386283101, end_angle=np.pi/1.95 - 0.386283101),layer="below"
        ),
        ##### two point left ######
        dict(
            type="path",path="M -220,89.47765084 L -80, 89.47765084  L -80, 224 "+ellipse_arc(a=-237.5, b=237.5, start_angle=0.386283101, end_angle=np.pi/1.95 - 0.386283101),layer="below"
        ),
        #####above three point ######
        dict(
            type="path",path="M -250,350 L -250, 550 L 250, 550 L 250, 350 L -250,350",layer="below"
        ),
    ]

    return zones


# **Dashboard code**

In [13]:
import pandas as pd
import numpy as np
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objs as go
from dash.dependencies import Input, Output, State, ALL
from dash import dash_table
import logging
import json

# Configure logging
logging.basicConfig(level=logging.DEBUG)

min_shot_distance = DA_shooting_stats['SHOT_DISTANCE'].min()
max_shot_distance = DA_shooting_stats['SHOT_DISTANCE'].max()

# Prepare dictionary with season, PPG, RPG, APG, and PIE
DA_stats = []
for index, row in deni_stats.iterrows():
    season = row['Season']
    ppg = row['PTS'] / row['GP'] if row['GP'] > 0 else 0
    rpg = row['REB'] / row['GP'] if row['GP'] > 0 else 0
    apg = row['AST'] / row['GP'] if row['GP'] > 0 else 0
    spg = row['STL'] / row['GP'] if row['GP'] > 0 else 0
    mpg = row['MIN'] if row['GP'] > 0 else 0
    pie = row['PIE']*100 if not pd.isnull(row['PIE']) else 0

    season_stats = {
        'season': season,
        'PPG': ppg,
        'RPG': rpg,
        'SPG': spg,
        'MPG': mpg,
        'APG': apg,
        'PIE': pie
    }
    DA_stats.append(season_stats)


# Create the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Add this block for the CSS
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
        <style>
            .custom-switch .form-check-input {
                width: 50px;
                height: 25px;
                background-color: #E31837;
                position: relative;
                box-shadow: inset 0 0 12px rgba(0, 0, 0, 0.2);
                border: 2px solid white;
            }

            .custom-switch .form-check-input:checked {
                background-color: #002B5C;
                border: 2px solid white;
            }

            .custom-switch .form-check-input::before {
                content: '';
                position: absolute;
                width: 21.5px;
                height: 21.5px;
                background-color: white;
                border-radius: 50%;
                transition: transform 0.2s;
                border: 1px solid black;
            }

            .custom-switch .form-check-input:checked::before {
                transform: translateX(25px);

            }

            .slider {
              position: absolute;
              cursor: pointer;
              top: 0;
              left: 0;
              right: 0;
              bottom: 0;
              background-color: #ccc;
              transition: .4s;
              border-radius: 34px;
            }

            .slider:before {
              position: absolute;
              content: "";
              height: 26px;
              width: 26px;
              left: 4px;
              bottom: 4px;
              background-color: white;
              transition: .4s;
              border-radius: 50%;
            }

            input:checked + .slider {
              background-color: #2196F3;
            }

            input:checked + .slider:before {
              transform: translateX(26px);
            }

            .switch-label {
              margin: 0 10px;
              font-family: 'Segoe UI';
              font-size: 18px;
            }

            .rc-slider-track {
                height:6px;
                background-color: #002B5C;
            }

            .rc-slider-handle {
                width: 20px;
                height: 20px;
                background-color: #E31837;
                border: 2px solid #000000;
            }

            .rc-slider-dot {
                width: 12px;
                height: 12px;
                background-color: #E31837;
                border: 2px solid #000000;
                bottom: -5px;
            }
            .custom-radio input[type="radio"] {
                -webkit-appearance: none;
                -moz-appearance: none;
                appearance: none;
                display: inline-block;
                width: 20px;
                height: 20px;
                padding: 3px;
                background-clip: content-box;
                border: 2px solid #002B5C;
                background-color: #002B5C;
                border-radius: 50%;
                vertical-align: middle;
            }

            .custom-radio input[type="radio"]:checked {
                background-color: #E31837;
                border-color: #E31837;
            }

            .custom-radio label {
                font-family: 'Segoe UI';
                font-size: 16px;
                margin-right: 20px;
                display: inline-block;
                padding: 9px 0;
                vertical-align: middle;  /* Add this line */
            }

            /* Add this new rule */
            .custom-radio input[type="radio"] + label {
                margin-left: 5px;  /* Space between radio button and its label */
            }

        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

# Get unique seasons
seasons = sorted(DA_shooting_stats['SEASON'].unique())

def generate_stats_table(selected_seasons):
    # Filter the stats based on selected seasons
    filtered_stats = [stat for stat in DA_stats if stat['season'] in selected_seasons]

    # Calculate mean of stats for selected seasons
    mean_stats = {
        'PPG': sum(stat['PPG'] for stat in filtered_stats) / len(filtered_stats),
        'RPG': sum(stat['RPG'] for stat in filtered_stats) / len(filtered_stats),
        'APG': sum(stat['APG'] for stat in filtered_stats) / len(filtered_stats),
        'PIE': sum(stat['PIE'] for stat in filtered_stats) / len(filtered_stats)
    }

    stats_to_display = ['PPG', 'RPG', 'APG', 'PIE']

    # Additional stats
    additional_stats = {
        'HEIGHT': "6'9\" (2.06m)",
        'WEIGHT': "210lb (95kg)",
        'COUNTRY': "Israel",
        'LAST ATTENDED': "Maccabi Tel Aviv",
        'AGE': "23 years",
        'BIRTHDATE': "January 3, 2001",
        'DRAFT': "2020 R1 Pick 9",
        'EXPERIENCE': "4 Years"
    }

    return html.Div([
        html.Div([
            html.H4('Season Averages', style={'textAlign': 'center', 'color': 'white', 'marginBottom': '8px'}),
            html.Table([
                html.Tr([
                    html.Td(html.Div([
                        html.Div(stat, style={'fontSize': '20px', 'fontFamily': 'Segoe UI', 'color': 'white'}),
                        html.Div(f"{mean_stats[stat]:.1f}", style={'fontSize': '24px', 'fontWeight': 'bold', 'color': 'white'})
                    ]), style={'padding': '10px', 'textAlign': 'center', 'border': '2.5px solid white', 'backgroundColor': '#001F40'})
                    for stat in stats_to_display
                ])
            ], style={'width': '100%', 'backgroundColor': '#001F40', 'padding': '10px', 'height': '100%'})
        ], style={'width': '45%', 'display': 'flex', 'flexDirection': 'column'}),

        html.Div([
            html.H4('Player Profile', style={'textAlign': 'center', 'color': 'white', 'marginBottom': '8px'}),
            html.Table([
                html.Tr([
                    html.Td(html.Div([
                        html.Div(key, style={'fontSize': '12px', 'fontFamily': 'Segoe UI', 'color': 'white'}),
                        html.Div(value, style={'fontSize': '14px', 'fontFamily': 'Segoe UI', 'fontWeight': 'bold', 'color': 'white'})
                    ]), style={'padding': '10px', 'textAlign': 'center', 'border': '2.5px solid white', 'backgroundColor': '#001F40'})
                    for key, value in list(additional_stats.items())[:4]
                ]),
                html.Tr([
                    html.Td(html.Div([
                        html.Div(key, style={'fontSize': '12px', 'fontFamily': 'Segoe UI', 'color': 'white'}),
                        html.Div(value, style={'fontSize': '14px', 'fontFamily': 'Segoe UI', 'fontWeight': 'bold', 'color': 'white'})
                    ]), style={'padding': '10px', 'textAlign': 'center', 'border': '2.5px solid white', 'backgroundColor': '#001F40'})
                    for key, value in list(additional_stats.items())[4:]
                ])
            ], style={'width': '100%', 'backgroundColor': '#001F40', 'padding': '10px', 'height': '100%'})
        ], style={'width': '55%', 'display': 'flex', 'flexDirection': 'column'}),
    ], style={'display': 'flex', 'flexDirection': 'row', 'alignItems': 'stretch', 'margin': '5px 0', 'overflow': 'hidden'})

# Define the layout
app.layout = dbc.Container([

    dbc.Row([
        # Left column for the image
        dbc.Col([
            html.Img(src="assets/deni11.jpg", style={
                'width': '90%',
                'height': '100vh',  # Full viewport height
                'object-fit': 'cover',
                'position': 'sticky',
                'top': '0',
            })
        ], width=2, style={'padding': 0}),  # Adjust width as needed

        # Right column for the dashboard content
        dbc.Col([
            html.Div([
                html.H1(
                    'Deni Avdija NBA Stats Dashboard',
                    style={
                        'textAlign': 'center',
                        'fontFamily': 'Segoe UI',
                        'marginBottom': '5px',
                        'backgroundColor': '#002B5C',  # Dark blue-gray color
                        'color': 'white',
                        'fontWeight': 'bold',
                        'padding': '20px',
                        'width': '100%',  # Ensure full width
                        'family':'Segoe UI'
                    }
                ),
                # New stats table section
                html.H5("Select seasons", style={'textAlign': 'center','marginBottom': '5px', 'color': 'white'}),
                html.Div([
                    html.Button('All Seasons', id='all-seasons-button', n_clicks=0,
                                style={'margin': '10px',
                                      'padding': '15px 60px',
                                      'fontSize': '18px',
                                      'fontWeight': 'bold',
                                      'family':'Segoe UI',
                                      'backgroundColor': '#2c3e50',  # Elegant dark color
                                      'color': 'white',
                                      'border': 'none',
                                      'borderRadius': '5px',
                                      'cursor': 'pointer',
                                      'transition': 'background-color 0.3s'}),
                    *[html.Button(season, id={'type': 'season-button', 'index': season}, n_clicks=0,
                                  style={'margin': '10px',
                                        'padding': '15px 60px',
                                        'fontSize': '16px',
                                         'family':'Segoe UI',
                                        'backgroundColor': '#2c3e50',  # Elegant dark color
                                        'border': 'none',
                                        'borderRadius': '5px',
                                        'color': 'white',
                                        'cursor': 'pointer',
                                        'transition': 'background-color 0.3s',

})
                      for season in seasons]
                ], style={'display': 'flex', 'flexWrap': 'wrap', 'justifyContent': 'center', 'alignItems': 'center', 'gap': '5px'}),
                html.Div(id='selected-seasons', style={'display': 'none'}),
                html.Div([
                  html.Div(id='stats-table', style={'width': '100%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '20px',  'background-color': '#002B5C'})
                    ], style={'padding': '5px', 'backgroundColor': '#002B5C'}),

            ], style={'backgroundColor': '#002B5C', 'padding': '2px','marginBottom': '20px','box-shadow': '0px 0px 10px 0px #002B5C'},

                     )    ,



            html.Div([
        html.Div([
            html.H2("Deni Avdija's Shooting Performance", style={'textAlign': 'center','marginBottom': '20px', 'fontFamily': 'Segoe UI'}),
            html.H6("Select Shot Visualization", style={'textAlign': 'left','fontFamily': 'Segoe UI'}),
            dcc.RadioItems(
                  id='shot-result-filter',
                  options=[
                      {'label': 'All', 'value': 'all'},
                      {'label': 'Made', 'value': 'made'},
                      {'label': 'Missed', 'value': 'missed'},
                      {'label': 'Hot Spots 🔥', 'value': 'hot_spots'},  # Added fire emoji here
                  ],
                  value='all',
                  className='custom-radio',
                  labelStyle={
                      'display': 'inline-block',
                      'margin-right': '50px',  # Increase this value for more space between options
                      'margin-bottom': '1px',  # Adds some vertical space
                      'font-family': 'Segoe UI',
                      'font-size': '16px',
                      'align-items': 'center',
                  },
                  inputStyle={
                      "margin-right": "5px",
                      "vertical-align": "middle",
                  },
                  style={
                      'display': 'flex',
                      'flex-wrap': 'wrap',
                      'align-items': 'center',
                      'justify-content': 'center',
                      'textAlign': 'center',
                      'fontFamily': 'Segoe UI',
                      'border': '1px solid #F0F0F0',
                      'border-radius': '5px',
                      'background-color': '#F0F0F0',
                      'padding': '0.1px',  # Increased padding
                      'marginBottom': '1px',
                      'marginLeft': '50px'
                  }
              ),

            dcc.Graph(
                id='scoring-chart',
                config={'displayModeBar': False},
                style={'marginBottom': '20px'}
            ),
          dcc.RangeSlider(
              id='shot-chart-range-slider',
              min=min_shot_distance,
              max=max_shot_distance,
              value=[min_shot_distance, max_shot_distance],  # Initial range based on min and max
              marks={0:{'label': 'Basket'},8:{'label': 'In Paint'},16: {'label': 'Mid-Range'},21.5: {'label': 'Three Point Line'}},  # Marks based on range
              # step=1,  # Step size of 1
              tooltip={
                      'always_visible': True,
                      'label': {
                          'prefix': 'Distance: ',
                          'style': {'color': 'white', 'fontSize': '14px', 'textAlign': 'center','family':'Segoe UI',}
                      }
    },
          ),

        ], style={'marginRight':'2%','width': '50%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '10px', 'border': '1px solid #e0e0e0', 'border-radius': '10px', 'background-color': '#F0F0F0','box-shadow': '0px 10px 10px 0px #C4CED4'
})
        ,
                    # Here is where you integrate the slider
                html.Div([
                    html.H2("Deni Avdija's Performance Rates", style={'textAlign': 'center', 'fontFamily': 'Segoe UI'}),
                    html.H2("& Players Comparison", style={'textAlign': 'center', 'fontFamily': 'Segoe UI'}),
                    dcc.Graph(
                        id='spider-plot',
                        config={'displayModeBar': False},
                        style={'height': '528px'}  # Adjust this value as needed
                    )
        ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '20px', 'border': '1px solid #e0e0e0', 'border-radius': '10px', 'background-color': '#F0F0F0','box-shadow': '0px 10px 10px 0px #C4CED4'})
            ]),
          html.Div([
                    ], style={'padding': '12px', 'backgroundColor': '#F0F0F0'}),
            html.Div([

                html.H2("Deni Avdija's Defensive/Offensive Performance", style={'textAlign': 'center', 'fontFamily': 'Segoe UI'}),
                html.Div([
                    html.Div("Defense", style={'flex': '1', 'textAlign': 'right', 'paddingRight': '10px'}),
                    dbc.Switch(
                        id='performance-mode-switch',
                        value=False,
                        className="custom-switch",
                        style={'margin': 'auto'}
                    ),
                    html.Div("Offense", style={'flex': '1', 'textAlign': 'left', 'paddingLeft': '10px'})
                ], style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 'padding': '15px'}),
                dcc.Graph(
                    id='performance-plot',
                    config={'displayModeBar': False}
                )
        ], style={'width': '100%', 'display': 'inline-block', 'verticalAlign': 'top', 'border': '1px solid #e0e0e0', 'border-radius': '10px', 'background-color': '#F0F0F0','box-shadow': '0px 0px 15px 0px #C4CED4'}),
        ], width=10),  # Adjust width as needed
    ], className='g-0')  # Remove gutters between columns
], fluid=True, style={'backgroundColor': '#F0F0F0'})


@app.callback(
    Output('stats-table', 'children'),
    [Input('selected-seasons', 'children')]
)
def update_stats_table(selected_seasons_json):
    selected_seasons = json.loads(selected_seasons_json)
    return generate_stats_table(selected_seasons)

# Callback to update selected seasons
@app.callback(
    [Output('selected-seasons', 'children'),
     Output({'type': 'season-button', 'index': ALL}, 'style'),
     Output('all-seasons-button', 'style'),
     Output({'type': 'season-button', 'index': ALL}, 'n_clicks')],
    [Input('all-seasons-button', 'n_clicks'),
     Input({'type': 'season-button', 'index': ALL}, 'n_clicks')],
    [State({'type': 'season-button', 'index': ALL}, 'id')]
)
def update_selected_seasons(all_clicks, season_clicks, season_ids):
    ctx = dash.callback_context
    if not ctx.triggered:
        selected = seasons
        all_button_style = {
            'margin': '10px',
            'padding': '15px 60px',
            'fontSize': '18px',
            'fontWeight': 'bold',
            'family':'Segoe UI',
            'backgroundColor': '#E31837',  # Elegant dark color
            'color': 'white',
            'border': 'none',
            'borderRadius': '5px',
            'cursor': 'pointer',
            'transition': 'background-color 0.3s',
        }
        season_styles = [{'margin': '10px',
                          'padding': '15px 60px',
                          'fontSize': '16px',
                          'family':'Segoe UI',
                          'backgroundColor': '#E31837',  # Elegant shooting color
                          'color': 'white',
                          'border': 'none',
                          'borderRadius': '5px',
                          'cursor': 'pointer',
                          'transition': 'background-color 0.3s'} for _ in seasons]
        season_clicks = [0] * len(seasons)  # Reset clicks for all season buttons
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        if button_id == 'all-seasons-button':
            selected = seasons
            all_button_style = {
                'margin': '10px',
                'padding': '15px 60px',
                'fontSize': '18px',
                'fontWeight': 'bold',
                'family':'Segoe UI',
                'backgroundColor': '#E31837',  # Elegant dark color
                'color': 'white',
                'border': 'none',
                'borderRadius': '5px',
                'cursor': 'pointer',
                'transition': 'background-color 0.3s'
            }
            season_styles = [{'margin': '10px',
                              'padding': '15px 60px',
                              'fontSize': '16px',
                              'family':'Segoe UI',
                              'backgroundColor': '#E31837',  # Elegant dark color
                              'color': 'white',
                              'border': 'none',
                              'borderRadius': '5px',
                              'cursor': 'pointer',
                              'transition': 'background-color 0.3s'} for _ in seasons]
            season_clicks = [0] * len(seasons)  # Reset clicks for all season buttons
        else:
            selected = [id['index'] for clicks, id in zip(season_clicks, season_ids) if clicks and clicks % 2 != 0]
            all_button_style = {
                'margin': '10px',
                'padding': '15px 60px',
                'fontSize': '18px',
                'fontWeight': 'bold',
                'family':'Segoe UI',
                'backgroundColor': '#C4CED4',
                'color': 'black',
                'border': 'none',
                'borderRadius': '5px',
                'cursor': 'pointer',
                'transition': 'background-color 0.3s'
            }
            season_styles = [{'margin': '10px',
                              'padding': '15px 60px',
                              'fontSize': '16px',
                              'family':'Segoe UI',
                              'backgroundColor': '#E31837' if season in selected else '#C4CED4',
                              'color': 'white' if season in selected else 'black',
                              'border': 'none',
                              'borderRadius': '5px',
                              'cursor': 'pointer',
                              'transition': 'background-color 0.3s'} for season in seasons]
            if set(selected) == set(seasons):
                all_button_style['backgroundColor'] = '#E31837'
                all_button_style['color'] = '#C4CED4'

    return json.dumps(selected), season_styles, all_button_style, season_clicks


# Modify existing callbacks to use the new selected seasons
@app.callback(
    Output('scoring-chart', 'figure'),
    [Input('selected-seasons', 'children'), Input('shot-result-filter', 'value'), Input('shot-chart-range-slider', 'value')]
)
def update_graph(selected_seasons_json, shot_result, slider_range):
    start_index, end_index = slider_range[0], slider_range[1]
    selected_seasons = json.loads(selected_seasons_json)

    # Filter data based on selected seasons
    filtered_data_by_season = DA_shooting_stats[DA_shooting_stats['SEASON'].isin(selected_seasons)]

    # Further filter by shot distance range
    filtered_data_by_season = filtered_data_by_season[(filtered_data_by_season['SHOT_DISTANCE'] >= start_index) & (filtered_data_by_season['SHOT_DISTANCE'] <= end_index)]

    fig = go.Figure()

    if shot_result == 'hot_spots':
        zones = create_zones()
        def point_in_polygon(point, polygon):
            x, y = point
            n = len(polygon)
            inside = False

            p1x, p1y = polygon[0]
            for i in range(n + 1):
                p2x, p2y = polygon[i % n]
                if y > min(p1y, p2y):
                    if y <= max(p1y, p2y):
                        if x <= max(p1x, p2x):
                            if p1y != p2y:
                                xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                            if p1x == p2x or x <= xinters:
                                inside = not inside
                p1x, p1y = p2x, p2y

            return inside

        # Function to extract coordinates from a path string
        def parse_path(path_str):
            path = []
            commands = re.findall(r'[MLQ]\s*[-+]?\d*\.?\d+\s*,\s*[-+]?\d*\.?\d*', path_str)
            for command in commands:
                parts = re.findall(r'[-+]?\d*\.?\d+', command)
                if len(parts) >= 2:
                    x = float(parts[0])
                    y = float(parts[1])
                    path.append((x, y))
            return path
        zones = create_zones()
        color_lst = []
        annotations = []

        for zone in zones:
            # Filter shots within the current zone
            path_str = zone['path']
            parsed_path = parse_path(path_str)
            mask = []
            for i, row in filtered_data_by_season.iterrows():
                point = (row['LOC_X'], row['LOC_Y'])
                if point_in_polygon(point, parsed_path):
                    mask.append(True)
                else:
                    mask.append(False)
            # Calculate shot statistics for the zone...
            # Example calculation:
            zone_data = filtered_data_by_season[mask]
            total_shots = len(zone_data)
            if total_shots == 0:
                  additional_row = pd.DataFrame({'LOC_X': [0], 'LOC_Y': [375]})
                  zone_data = pd.concat([zone_data, additional_row], ignore_index=True)
            mean_x = zone_data['LOC_X'].mean()
            mean_y = zone_data['LOC_Y'].mean()
            if zone_data['LOC_Y'].mean() > 420:
              mean_y = 375

            made_shots = zone_data['SHOT_MADE_FLAG'].sum()
            percentage = made_shots / total_shots if total_shots > 0 else 0

            if total_shots == 0:
                # If no shots in the zone, use a transparent color
                color = "rgba(0, 0, 0, 0)"  # Completely transparent
            else:
                made_shots = zone_data['SHOT_MADE_FLAG'].sum()
                percentage = made_shots / total_shots

                # Determine color based on percentage
                color = "rgba(255, 0, 0, 0.9)" if percentage <= 0.2 else \
                        "rgba(255, 69, 0, 0.9)" if percentage <= 0.25 else \
                        "rgba(255, 140, 0, 0.9)" if percentage <= 0.32 else \
                        "rgba(173, 255, 47, 0.9)" if percentage <= 0.36 else \
                        "rgba(127, 255, 0, 0.9)" if percentage <= 0.42 else \
                        "rgba(0, 255, 127, 0.9)" if percentage <= 0.5 else \
                        "rgba(0, 255, 0, 0.9)"

            color_lst.append(color)


            if total_shots > 0:
                annotations.append({
                    'x': mean_x,
                    'y': mean_y,
                    'text': f"{made_shots}/{total_shots}<br>{percentage:.2%}"
                })

        # Add annotations to the figure
        for ann in annotations:
            fig.add_annotation(
                x=ann['x'],
                y=ann['y'],
                text=ann['text'],
                showarrow=False,
                font=dict(size=10, color='white', family='Segoe UI'),
                align='center',
                bgcolor='rgba(0,0,0,0.5)'
            )

        def ellipse_arc(x_center=0.0, y_center=0.0, a=10.5, b=10.5, start_angle=0.0, end_angle=2 * np.pi, N=200, closed=False):
            t = np.linspace(start_angle, end_angle, N)
            x = x_center + a * np.cos(t)
            y = y_center + b * np.sin(t)
            path = f'M {x[0]}, {y[0]}'
            for k in range(1, len(t)):
                path += f'L{x[k]}, {y[k]}'
            if closed:
                path += ' Z'
            return path


        performance_levels = ["0%", "20%", "25%", "32%", "36%", "42%", "50%"]
        colorscale = [
            [0.0, "rgba(255, 0, 0, 0.6)"],      # <= 20%
            [0.2, "rgba(255, 0, 0, 0.6)"],      # 20%
            [0.25, "rgba(255, 69, 0, 0.6)"],    # <= 25%
            [0.32, "rgba(255, 140, 0, 0.6)"],   # <= 32%
            [0.36, "rgba(173, 255, 47, 0.6)"],  # <= 36%
            [0.42, "rgba(127, 255, 0, 0.6)"],   # <= 42%
            [0.5, "rgba(0, 255, 127, 0.6)"],    # <= 50%
            [1.0, "rgba(0, 255, 0, 0.6)"]       # 100%
        ]

        # Create a horizontal color scale marker above the plot
        fig.add_trace(go.Scatter(
            x=[None],
            y=[None],
            mode='markers',
            name='Performance Legend',
            marker=dict(
                size=0,
                color=[0.5],  # Use a middle value for the color (e.g., 0.5)
                colorscale=colorscale,
                showscale=True,
                colorbar=dict(
                    title='Shooting<br> Percentage',
                    tickvals=[0,0.2, 0.3, 0.4, 0.5, 0.6, 0.8],
                    ticktext=performance_levels,
                    len=0.85,  # Adjust length of the color bar relative to the plot
                    ticks='outside',
                    ticklen=8,
                    tickfont=dict(size=10, family='Segoe UI'),  # Adjust the font size here (smaller)
                    titlefont=dict(size=12, family='Segoe UI'),  # Adjust the title font size here
                    x=1,  # Position the color bar in the middle of the plot horizontally
                    xanchor='center',  # Anchor point of x coordinate (center)
                    y=1,  # Shift the color bar above the plot (increase or decrease as needed)
                    yanchor='top',  # Anchor point of y coordinate (top side)
                    # orientation='h',  # Make the color bar horizontal
                    xpad=8,
                ),
            ),
        ))

        threept_break_y = 89.47765084
        three_line_col = "#000000"
        main_line_col = "#000000"
        with open("court.jpeg", "rb") as image_file:
            encoded_image = base64.b64encode(image_file.read()).decode('ascii')
        fig.add_layout_image(
            dict(
                source=f"data:image/jpeg;base64,{encoded_image}",
                xref="x",
                yref="y",
                x=250,  # Changed from -250 to 250
                y=-52.5,  # Changed from 417.5 to -52.5
                sizex=500,
                sizey=470,
                sizing="stretch",
                opacity=0.7,
                layer="below",
                xanchor="right",  # Anchor to right side
                yanchor="bottom",  # Anchor to bottom
            )
        )
        line_width = 2
        fig.update_layout(
            # Line Horizontal
            margin=dict(l=20, r=20, t=24, b=20),
            paper_bgcolor="#F0F0F0",
            plot_bgcolor="#F0F0F0",
            yaxis=dict(
                scaleanchor="x",
                showgrid=False,
                zeroline=False,
                showline=False,
                ticks='',
                showticklabels=False,
                fixedrange=True,
            ),
            xaxis=dict(
                showgrid=False,
                zeroline=False,
                showline=False,
                ticks='',
                showticklabels=False,
                fixedrange=True,
            ),
          shapes=[
            dict(
                type="rect", x0=-250, y0=-52.5, x1=250, y1=417.5
            ),
            dict(
                type="rect", x0=-80, y0=-52.5, x1=80, y1=137.5
            ),
            dict(
                type="rect", x0=-60, y0=-52.5, x1=60, y1=137.5
            ),
            dict(
                type="circle", x0=-60, y0=77.5, x1=60, y1=197.5, xref="x", yref="y"
            ),
            dict(
                type="line", x0=-60, y0=137.5, x1=60, y1=137.5
            ),
            dict(
                type="rect", x0=-2, y0=-7.25, x1=2, y1=-12.5,
                fillcolor='black',
            ),
            dict(
                type="circle", x0=-7.5, y0=-7.5, x1=7.5, y1=7.5, xref="x", yref="y"
            ),
            dict(
                type="line", x0=-30, y0=-12.5, x1=30, y1=-12.5
            ),
            dict(type="path",
                  path=ellipse_arc(a=40, b=40, start_angle=0, end_angle=np.pi)),
            dict(
                type="line", x0=-220, y0=-52.5, x1=-220, y1=threept_break_y
            ),
            ######## three point right ######
            dict(type="path",
                  path=ellipse_arc(a=237.5, b=237.5, start_angle=0.386283101, end_angle=np.pi/1.95 - 0.386283101)+' L 80,350'+' L 250,350'+'L 250, 89.47765084', fillcolor=color_lst[0],layer="below"),
            ####### three point left ########
            dict(type="path",
                  path=ellipse_arc(a=-237.5, b=237.5, start_angle=0.386283101, end_angle=np.pi/1.95 - 0.386283101) +' L -80,350'+' L -250,350'+'L -250, 89.47765084', fillcolor=color_lst[1],layer="below"),
            ##### three point center ######
        dict(
            type="path",
            path=create_quadratic_bezier_curve(
                215.5 * np.cos(np.pi/2 - 0.386283101), 241 * np.sin(np.pi/2 - 0.386283101),
                -210.5 * np.cos(np.pi/2 - 0.386283101), 241 * np.sin(np.pi/2 - 0.386283101),
                curvature=0.3  # Adjust curvature as needed
            ) +' L -80,350' + ' L 80,350'+' L 80,224',fillcolor=color_lst[2],layer="below"
        ),
            ##### three point right corner ######
            dict(
                type="path",path="M -220,-52.5 L -220, 89.47765084 L -250, 89.47765084 L -250, -52.5 L -220, -52.5 ", fillcolor=color_lst[3],layer="below"
            ),
            ##### three point left corner ######
          dict(
                type="path",path="M 220,-52.5 L 220, 89.47765084 L 250, 89.47765084 L 250, -52.5 L 220, -52.5 ", fillcolor=color_lst[4],layer="below"
            ),
            ##### two point right corner ######
            dict(
                type="path",path="M 220,-52.5 L 80, -52.5  L 80, 89.47765084  L 220, 89.47765084 L 220,-52.5", fillcolor=color_lst[5],layer="below"
            ),
            ##### two point left corner ######
            dict(
                type="path",path="M -220,-52.5 L -80, -52.5  L -80, 89.47765084  L -220, 89.47765084 L -220,-52.5", fillcolor=color_lst[6],layer="below"
            ),
            ##### two point center ######
            dict(
                type="path",path=create_quadratic_bezier_curve(
                215.5 * np.cos(np.pi/2 - 0.386283101), 241 * np.sin(np.pi/2 - 0.386283101),
                -215.5 * np.cos(np.pi/2 - 0.386283101), 241 * np.sin(np.pi/2 - 0.386283101),
                curvature=0.3  # Adjust curvature as needed
            ) + 'L -80, 89.47765084 L 80, 89.47765084 L 80, 224' , fillcolor=color_lst[7],layer="below"
            ),
            ##### in paint ######
            dict(
                type="path",path="M -80,-52.5 L 80, -52.5  L 80, 89.47765084  L -80, 89.47765084 L -80, -52.5 ", fillcolor=color_lst[8],layer="below"
            ),
            ##### two point right ######
            dict(
                type="path",path="M 220,89.47765084 L 80, 89.47765084  L 80, 224 "+ellipse_arc(a=237.5, b=237.5, start_angle=0.386283101, end_angle=np.pi/1.95 - 0.386283101), fillcolor=color_lst[9],layer="below"
            ),
            ##### two point left ######
            dict(
                type="path",path="M -220,89.47765084 L -80, 89.47765084  L -80, 224 "+ellipse_arc(a=-237.5, b=237.5, start_angle=0.386283101, end_angle=np.pi/1.95 - 0.386283101), fillcolor=color_lst[10],layer="below"
            ),
            #####above three point ######
            dict(
                type="path",path="M -250,350 L -250, 417.5 L 250, 417.5 L 250, 350 L -250,350", fillcolor=color_lst[11],layer="below"

            ),
            dict(
                type="line", x0=-220, y0=-52.5, x1=-220, y1=threept_break_y
            ),
            dict(
                type="line", x0=220, y0=-52.5, x1=220, y1=threept_break_y
            ),
            dict(
                type="line", x0=-250, y0=227.5, x1=-220, y1=227.5
            ),
            dict(
                type="line", x0=250, y0=227.5, x1=220, y1=227.5
            ),
            dict(
                type="line", x0=-90, y0=17.5, x1=-80, y1=17.5
            ),
            dict(
                type="line", x0=-90, y0=27.5, x1=-80, y1=27.5
            ),
            dict(
                type="line", x0=-90, y0=57.5, x1=-80, y1=57.5
            ),
            dict(
                type="line", x0=-90, y0=87.5, x1=-80, y1=87.5
            ),
            dict(
                type="line", x0=90, y0=17.5, x1=80, y1=17.5
            ),
            dict(
                type="line", x0=90, y0=27.5, x1=80, y1=27.5
            ),
            dict(
                type="line", x0=90, y0=57.5, x1=80, y1=57.5
            ),
            dict(
                type="line", x0=90, y0=87.5, x1=80, y1=87.5
            ),
            dict(type="path",
                  path=ellipse_arc(y_center=417.5, a=60, b=60, start_angle=-0, end_angle=-np.pi)
                              ),
        ]
        )


    else:
        # Further filter the data based on the shot result
        if shot_result == 'made':
            filtered_data_by_season = filtered_data_by_season[filtered_data_by_season['SHOT_MADE_FLAG'] == 1]
        elif shot_result == 'missed':
            filtered_data_by_season = filtered_data_by_season[filtered_data_by_season['SHOT_MADE_FLAG'] == 0]

        # Made shots
        made_shots = filtered_data_by_season[filtered_data_by_season['SHOT_MADE_FLAG'] == 1]
        # Missed shots
        missed_shots = filtered_data_by_season[filtered_data_by_season['SHOT_MADE_FLAG'] == 0]

        # Add made shots
        fig.add_trace(go.Scatter(
            x=made_shots['LOC_X'],
            y=made_shots['LOC_Y'],
            mode='markers',
            name='Made Shot',
            marker=dict(color='green', size=8, line=dict(color='black', width=1)),
            text=made_shots.apply(lambda row: f"Shot Type: {row['SHOT_TYPE']}<br>Shot Action: {row['ACTION_TYPE']}<br>Shot Zone: {row['SHOT_ZONE_BASIC']}<br>Distance: {row['SHOT_DISTANCE']} ft", axis=1),
            hoverinfo='text'

        ))

        # Add missed shots
        fig.add_trace(go.Scatter(
            x=missed_shots['LOC_X'],
            y=missed_shots['LOC_Y'],
            mode='markers',
            name='Missed Shot',
            marker=dict(color='#E31837', size=8, symbol='x', line=dict(color='black', width=1)),
            text=missed_shots.apply(lambda row: f"Shot Type: {row['SHOT_TYPE']}<br>Shot Action: {row['ACTION_TYPE']}<br>Shot Zone: {row['SHOT_ZONE_BASIC']}<br>Distance: {row['SHOT_DISTANCE']} ft", axis=1),
            hoverinfo='text'
        ))

        def ellipse_arc(x_center=0.0, y_center=0.0, a=10.5, b=10.5, start_angle=0.0, end_angle=2 * np.pi, N=200, closed=False):
              t = np.linspace(start_angle, end_angle, N)
              x = x_center + a * np.cos(t)
              y = y_center + b * np.sin(t)
              path = f'M {x[0]}, {y[0]}'
              for k in range(1, len(t)):
                  path += f'L{x[k]}, {y[k]}'
              if closed:
                  path += ' Z'
              return path

        threept_break_y = 89.47765084
        three_line_col = "#000000"
        main_line_col = "#000000"


        with open("court.jpeg", "rb") as image_file:
            encoded_image = base64.b64encode(image_file.read()).decode('ascii')
        fig.add_layout_image(
            dict(
                source=f"data:image/jpeg;base64,{encoded_image}",
                xref="x",
                yref="y",
                x=250,  # Changed from -250 to 250
                y=-52.5,  # Changed from 417.5 to -52.5
                sizex=500,
                sizey=470,
                sizing="stretch",
                opacity=0.7,
                layer="below",
                xanchor="right",  # Anchor to right side
                yanchor="bottom",  # Anchor to bottom
            )
        )
        fig_width = 3
        fig.update_layout(
            # Line Horizontal
            margin=dict(l=20, r=20, t=24, b=20),
            paper_bgcolor="#F0F0F0",
            plot_bgcolor="#F0F0F0",
            yaxis=dict(
                scaleanchor="x",
                showgrid=False,
                zeroline=False,
                showline=False,
                ticks='',
                showticklabels=False,
                fixedrange=True,
            ),
            xaxis=dict(
                showgrid=False,
                zeroline=False,
                showline=False,
                ticks='',
                showticklabels=False,
                fixedrange=True,
            ),
            shapes=[
                dict(
                    type="rect", x0=-250, y0=-52.5, x1=250, y1=417.5,
                    line=dict(color=main_line_col, width=fig_width),
                    # fillcolor='#333333',
                    layer='below'
                ),
                dict(
                    type="rect", x0=-80, y0=-52.5, x1=80, y1=137.5,
                    line=dict(color=main_line_col, width=fig_width),
                    # fillcolor='#333333',
                    layer='below'
                ),
                dict(
                    type="rect", x0=-60, y0=-52.5, x1=60, y1=137.5,
                    line=dict(color=main_line_col, width=fig_width),
                    # fillcolor='#333333',
                    layer='below'
                ),
                dict(
                    type="circle", x0=-60, y0=77.5, x1=60, y1=197.5, xref="x", yref="y",
                    line=dict(color=main_line_col, width=fig_width),
                    # fillcolor='#dddddd',
                    layer='below'
                ),
                dict(
                    type="line", x0=-60, y0=137.5, x1=60, y1=137.5,
                    line=dict(color=main_line_col, width=fig_width),
                    layer='below'
                ),

                dict(
                    type="rect", x0=-2, y0=-7.25, x1=2, y1=-12.5,
                    line=dict(color="#ec7607", width=fig_width),
                    fillcolor='#ec7607',
                ),
                dict(
                    type="circle", x0=-7.5, y0=-7.5, x1=7.5, y1=7.5, xref="x", yref="y",
                    line=dict(color="#ec7607", width=fig_width),
                ),
                dict(
                    type="line", x0=-30, y0=-12.5, x1=30, y1=-12.5,
                    line=dict(color="#ec7607", width=fig_width),
                ),

                dict(type="path",
                    path=ellipse_arc(a=40, b=40, start_angle=0, end_angle=np.pi),
                    line=dict(color=main_line_col, width=fig_width), layer='below'),
                dict(type="path",
                    path=ellipse_arc(a=237.5, b=237.5, start_angle=0.386283101, end_angle=np.pi - 0.386283101),
                    line=dict(color=main_line_col, width=fig_width), layer='below'),
                dict(
                    type="line", x0=-220, y0=-52.5, x1=-220, y1=threept_break_y,
                    line=dict(color=three_line_col, width=fig_width), layer='below'
                ),
                dict(
                    type="line", x0=-220, y0=-52.5, x1=-220, y1=threept_break_y,
                    line=dict(color=three_line_col, width=fig_width), layer='below'
                ),
                dict(
                    type="line", x0=220, y0=-52.5, x1=220, y1=threept_break_y,
                    line=dict(color=three_line_col, width=fig_width), layer='below'
                ),

                dict(
                    type="line", x0=-250, y0=227.5, x1=-220, y1=227.5,
                    line=dict(color=main_line_col, width=fig_width), layer='below'
                ),
                dict(
                    type="line", x0=250, y0=227.5, x1=220, y1=227.5,
                    line=dict(color=main_line_col, width=fig_width), layer='below'
                ),
                dict(
                    type="line", x0=-90, y0=17.5, x1=-80, y1=17.5,
                    line=dict(color=main_line_col, width=fig_width), layer='below'
                ),
                dict(
                    type="line", x0=-90, y0=27.5, x1=-80, y1=27.5,
                    line=dict(color=main_line_col, width=fig_width), layer='below'
                ),
                dict(
                    type="line", x0=-90, y0=57.5, x1=-80, y1=57.5,
                    line=dict(color=main_line_col, width=fig_width), layer='below'
                ),
                dict(
                    type="line", x0=-90, y0=87.5, x1=-80, y1=87.5,
                    line=dict(color=main_line_col, width=fig_width), layer='below'
                ),
                dict(
                    type="line", x0=90, y0=17.5, x1=80, y1=17.5,
                    line=dict(color=main_line_col, width=fig_width), layer='below'
                ),
                dict(
                    type="line", x0=90, y0=27.5, x1=80, y1=27.5,
                    line=dict(color=main_line_col, width=fig_width), layer='below'
                ),
                dict(
                    type="line", x0=90, y0=57.5, x1=80, y1=57.5,
                    line=dict(color=main_line_col, width=fig_width), layer='below'
                ),
                dict(
                    type="line", x0=90, y0=87.5, x1=80, y1=87.5,
                    line=dict(color=main_line_col, width=fig_width), layer='below'
                ),

                dict(type="path",
                    path=ellipse_arc(y_center=417.5, a=60, b=60, start_angle=-0, end_angle=-np.pi),
                    line=dict(color=main_line_col, width=fig_width), layer='below'),

            ],
            legend=dict(
                    x=0.95,
                    y=0.95,
                    xanchor='right',
                    yanchor='top',
                    bgcolor='rgba(255, 255, 255, 0.7)',
                    bordercolor='rgba(0, 0, 0, 0.5)',
                    borderwidth=1
                )
        )
    return fig

# Callback for spider plot

# @app.callback(
#     Output('spider-plot', 'figure'),
#     [Input('selected-seasons', 'children')],
#     [State('spider-plot', 'figure')]
# )
# def update_spider_plot(selected_seasons_json, current_figure):
#     selected_seasons = json.loads(selected_seasons_json)

#     # Process data
#     deni_data = get_data(deni_stats)
#     lebron_data = get_data(lebron_stats)
#     casspi_data = get_data(casspi_stats)

#     deni_filtered = deni_data[deni_data['Season'].isin(selected_seasons)][parameters]
#     lebron_filtered = lebron_data[lebron_data['Season'].isin(selected_seasons)][parameters]
#     casspi_filtered = casspi_data[casspi_data['Season'].isin(selected_seasons)][parameters]

#     deni_avg = deni_filtered.mean().to_dict()
#     lebron_avg = lebron_filtered.mean().to_dict()
#     casspi_avg = casspi_filtered.mean().to_dict()

#     # Get current visibility states
#     visibility = {
#         'Deni': True,
#         'LeBron': 'legendonly',
#         'Casspi': 'legendonly'
#     }
#     if current_figure is not None and 'data' in current_figure:
#         for trace in current_figure['data']:
#             if 'name' in trace and trace['name'] in visibility:
#                 visibility[trace['name']] = trace['visible']

#     # Create new figure
#     fig = go.Figure()

#     # Calculate max value for scaling
#     max_value = max(max(deni_avg.values()), max(lebron_avg.values()), max(casspi_avg.values()))
#     axis_range = [0, max(100, max_value * 1.1)]

#     # Function to add player data
#     def add_player_data(player_data, name, color):
#         categories = list(player_data.keys())
#         categories = [cat.replace(' ', '<br>') if len(cat) > 10 else cat for cat in categories]
#         values = list(player_data.values())
#         values_closed = values + [values[0]]

#         fig.add_trace(go.Scatterpolar(
#             r=values_closed,
#             theta=categories + [categories[0]],
#             name=name,
#             fill='toself',
#             fillcolor=f'rgba{(*hex_to_rgb(color), 0.2)}',
#             line=dict(color=color, width=2, shape='spline'),
#             hoverinfo='skip',
#             visible=visibility[name],
#             legendgroup=name,
#             opacity=0.8
#         ))
#         fig.add_trace(go.Scatterpolar(
#             r=values,
#             theta=categories,
#             mode='markers',
#             name=name,
#             marker=dict(
#                 color=color,
#                 size=8,
#                 symbol='circle',
#                 line=dict(color='white', width=1)
#             ),
#             text=[f"{name}<br>{cat}: {val:.2f}" for cat, val in zip(categories, values)],
#             hoverinfo='text',
#             showlegend=False,
#             visible=visibility[name],
#             legendgroup=name
#         ))

#     # Add data for all players
#     add_player_data(deni_avg, 'Deni', '#002B5C')
#     add_player_data(lebron_avg, 'LeBron', '#FDB927')
#     add_player_data(casspi_avg, 'Casspi', '#5A2D81')

#     fig.update_layout(
#         polar=dict(
#             radialaxis=dict(
#                 visible=True,
#                 range=axis_range,
#                 color='gray',
#                 gridcolor='lightgray'
#             ),
#             angularaxis=dict(
#                 gridcolor='lightgray'
#             )
#         ),
#         showlegend=True,
#         legend=dict(
#             title=dict(text='Add Players', font=dict(size=18, color='#333', family='Segoe UI')),
#             traceorder='reversed',
#             itemsizing='constant',
#             bgcolor='rgba(255, 255, 255, 0.9)',
#             bordercolor='#999',
#             borderwidth=1,
#             font=dict(size=14, color='#333', family='Segoe UI'),
#             itemclick='toggle',
#             itemdoubleclick='toggleothers',
#             x=1.2,
#             y=0.95,
#             xanchor='right',
#             yanchor='top',
#             orientation='v',
#             itemwidth=30,
#             # itemsizing='constant',
#         ),
#         paper_bgcolor='#F8F8F8',
#         plot_bgcolor='#F8F8F8',
#         margin=dict(l=80, r=100, t=40, b=80)
#     )

#     # Add annotation at the bottom
#     fig.add_annotation(
#         x=0.5,
#         y=-0.15,
#         xref='paper',
#         yref='paper',
#         text="* Omri Casspi's data was taken from his first 4 seasons",
#         showarrow=False,
#         font=dict(
#             family="Segoe UI",
#             size=12,
#             color="#666"
#         ),
#     )

#     return fig


@app.callback(
    Output('spider-plot', 'figure'),
    [Input('selected-seasons', 'children')],
    [State('spider-plot', 'figure')]
)
def update_spider_plot(selected_seasons_json, current_figure):
    selected_seasons = json.loads(selected_seasons_json)

    # Process data
    deni_data = get_data(deni_stats)
    lebron_data = get_data(lebron_stats)
    casspi_data = get_data(casspi_stats)

    deni_filtered = deni_data[deni_data['Season'].isin(selected_seasons)][parameters]
    lebron_filtered = lebron_data[lebron_data['Season'].isin(selected_seasons)][parameters]
    casspi_filtered = casspi_data[casspi_data['Season'].isin(selected_seasons)][parameters]

    deni_avg = deni_filtered.mean().to_dict()
    lebron_avg = lebron_filtered.mean().to_dict()
    casspi_avg = casspi_filtered.mean().to_dict()


    # Get current visibility states
    visibility = {
        'Deni': True,
        'LeBron': 'legendonly',
        'Casspi': 'legendonly'
    }
    if current_figure is not None and 'data' in current_figure:
        for trace in current_figure['data']:
            if 'name' in trace and trace['name'] in visibility:
                visibility[trace['name']] = trace['visible']

    # Sort categories based on Deni's values
    sorted_categories = sorted(deni_avg.keys(), key=lambda x: deni_avg[x], reverse=True)

    # Create new figure
    fig = go.Figure()

    # Function to add player data
    def add_player_data(player_data, name, color):
        values = [player_data[cat] for cat in sorted_categories]
        fig.add_trace(go.Bar(
            x=sorted_categories,
            y=values,
            name=name,
            marker_color=color,
            visible=visibility[name],
            text=[f"{val:.2f}" for val in values],
            textposition='auto',
            hoverinfo='text',
            hovertext=[f"{name}<br>{cat}: {val:.2f}" for cat, val in zip(sorted_categories, values)]
        ))

    # Add data for all players
    add_player_data(deni_avg, 'Deni', '#002B5C')
    add_player_data(lebron_avg, 'LeBron', '#FDB927')
    add_player_data(casspi_avg, 'Casspi', '#cf4dff')

    fig.update_layout(
        barmode='group',
        xaxis_title=dict(text='skills', font=dict(size=14, family='Segoe UI', color='#333')),
        yaxis_title=dict(text='Values', font=dict(size=16, family='Segoe UI', color='#333')),
        legend=dict(
            title=dict(text='Select players', font=dict(size=16, color='#333', family='Segoe UI')),
            traceorder='normal',
            bgcolor='rgba(240, 240, 240, 0.9)',  # Updated: semi-transparent light gray background
            font=dict(size=14, color='#333', family='Segoe UI'),
            orientation='h',  # Horizontal orientation
            yanchor='top',
            y=1.1,  # Position just above the plot area
            xanchor='left',
            x=0,
        ),
        margin=dict(l=60, r=30, t=30, b=60),  # Adjusted margins
        height=500,  # Set a fixed height
        plot_bgcolor='#F0F0F0',  # Plot background color with transparency
        paper_bgcolor='#F0F0F0',
    )

    # Adjust x-axis
    fig.update_xaxes(tickangle=-45, tickfont=dict(size=10))

    # Add annotation at the bottom
    fig.add_annotation(
        x=1.0,
        y=-0.35,
        xref='paper',
        yref='paper',
        text="* Omri Casspi's data was taken<br> from his first 4 seasons",
        showarrow=False,
        font=dict(
            family="Segoe UI",
            size=10,
            color="#666"
        ),
    )

    return fig



@app.callback(
    Output('performance-plot', 'figure'),
    [Input('selected-seasons', 'children'), Input('performance-mode-switch', 'value')]
)

def update_performance_plot(selected_seasons_json, mode):
    selected_seasons = json.loads(selected_seasons_json)

    # Filter data based on selected seasons
    filtered_df = DA_defence_offence_df[DA_defence_offence_df['SEASON'].isin(selected_seasons)]

    if mode:  # Offense mode
        grouped_df = filtered_df.groupby('GAME_NUMBER').agg({
            'offensiveRating': 'mean',
            'pace': 'mean',
            'effectiveFieldGoalPercentage': 'mean',
            'trueShootingPercentage': 'mean'
        }).reset_index()
        metrics = [
            ('offensiveRating', 'Offensive Rating', '#002B5C'),  # Dark blue
            ('pace', 'Pace', '#00509E'),  # Blue
            ('effectiveFieldGoalPercentage', 'eFG%', '#0074D9'),  # Light blue
            ('trueShootingPercentage', 'TS%', '#7FDBFF')  # Lighter blue
        ]
        title = "Values"

    else:  # Defense mode
        grouped_df = filtered_df.groupby('GAME_NUMBER').agg({
            'STL': 'mean',
            'defensiveRating': 'mean',
            'BLK': 'mean'
        }).reset_index()
        metrics = [
            ('STL', 'Steals', '#E31837'),  # Red
            ('defensiveRating', 'Defensive Rating', '#FFC300'),  # Light red
            ('BLK', 'Blocks', '#FFB3B3'),  # Pink
        ]
        title = "Values"

    # Create the figure
    fig = go.Figure()

    # Calculate moving average window size
    window_size = 14

    # Add traces for each metric with smoothed lines and colored lines
    for column, name, color in metrics:
        game_numbers = grouped_df['GAME_NUMBER'].values
        # Moving average
        ma = grouped_df[column].rolling(window=window_size, center=True).mean()
        # Moving average trace
        ma_trace = go.Scatter(
            x=game_numbers,
            y=ma,
            mode='lines',
            name=name,
            line=dict(color=color, width=2),  # Removed 'dash' property
            hoverinfo='none',
            # legendgroup=f'group_{yaxis}',  # Separate legend groups for y1 and y2
            legendgrouptitle_text=title ,
        )
        fig.add_trace(ma_trace)

    # Define season markers as vertical lines
    season_markers = [
        dict(
            type='line',
            x0=20, y0=0, x1=20, y1=1,
            xref='x', yref='paper',
            line=dict(color='rgba(65, 105, 225, 0.8)', width=1.5, dash='dash'),
        ),
        dict(
            type='line',
            x0=41, y0=0, x1=41, y1=1,
            xref='x', yref='paper',
            line=dict(color='rgba(220, 20, 60, 0.8)', width=1.5, dash='dash'),
        ),
        dict(
            type='line',
            x0=55, y0=0, x1=55, y1=1,
            xref='x', yref='paper',
            line=dict(color='rgba(148, 0, 211, 0.8)', width=1.5, dash='dash'),
        ),
        dict(
            type='line',
            x0=62, y0=0, x1=62, y1=1,
            xref='x', yref='paper',
            line=dict(color='rgba(60, 179, 113, 0.8)', width=1.5, dash='dash'),
        ),
    ]

    # Define text annotations for the lines
# Define text annotations for the lines
    text_annotations = [
        dict(x=21.5, y=0.01, xref='x', yref='paper', text='<b>1/4<br>Season</b>', showarrow=False,
            font=dict(size=10, color="rgba(0,0,0,0.8)", family="Segoe UI"), textangle=0),
        dict(x=39.5, y=0.01, xref='x', yref='paper', text='<b>Mid<br>Season</b>', showarrow=False,
            font=dict(size=10, color="rgba(0,0,0,0.8)", family="Segoe UI"), textangle=0),
        dict(x=53.4, y=0.01, xref='x', yref='paper', text='<b>All-Star<br>Break</b>', showarrow=False,
            font=dict(size=10, color="rgba(0,0,0,0.8)", family="Segoe UI"), textangle=0),
        dict(x=60.5, y=0.01, xref='x', yref='paper', text='<b>3/4<br>Season</b>', showarrow=False,
            font=dict(size=10, color="rgba(0,0,0,0.8)", family="Segoe UI"), textangle=0),
    ]

    # Update layout
    # Update layout
    fig.update_layout(
        xaxis_title=dict(text="Game Number", font=dict(size=16, family="Segoe UI", color="#555555")),
        yaxis_title=dict(text=title, font=dict(size=16, family="Segoe UI", color="#555555")),
        legend=dict(
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="#CCCCCC",
            borderwidth=1,
            font=dict(family="Segoe UI", size=12),
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01,
            traceorder='grouped'
        ),
        height=600,  # Increase the height of the graph
        margin=dict(l=60, r=60, t=20, b=80),  # Adjust the bottom margin to reduce space
        annotations=text_annotations,
        shapes=season_markers,  # Add the vertical lines
        xaxis=dict(
            showgrid=True,
            gridcolor='#E5E5E5',
            tickmode='linear',
            range=[1, 82],
            tickfont=dict(family="Segoe UI", size=10, color="#555555")
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='#E5E5E5',
            tickfont=dict(family="Segoe UI", size=12, color="#555555")
        ),
        plot_bgcolor='rgba(240,240,240,0.3)',
        paper_bgcolor='rgba(240,240,240,0.3)',
        hovermode='x unified'
    )

    # Update axes
    fig.update_xaxes(range=[1, 82], tickmode='linear', tick0=1, dtick=5)
    fig.update_yaxes(tickmode='auto', nticks=10)

    return fig





# Helper function to convert hex color to RGB
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))


# Run the app
if __name__ == '__main__':
    output.serve_kernel_port_as_window(8050)
    app.run_server(debug=False)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>